In [1]:
import numpy as np

from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
# read in data
cardtext = [list(x) for x in list(np.load('data/card_texts.npy'))]
c2i = np.load('data/c2i.npy').item()
i2c = np.load('data/i2c.npy').item()

In [3]:
# test - randomize!
np.random.seed = 1337
indices = list(np.random.permutation(len(cardtext)))
cardtext = [cardtext[i] for i in indices]
# cardtext = cardtext

In [4]:
# set parameters

DROP_RATE = 0.33 # dropout
EMBEDDING_SIZE = 256 # embedding size
HIDDEN_SIZE = 256 # lstm feature vector
HIDDEN_LAYERS = 2 # number of layers
START_EPOCH = 0
VOCAB_SIZE = len(c2i.keys()) # number of characters

WINDOW_SIZE = 20 # context length
CARDS_PER_BATCH = 10
NUM_EPOCHS = 1000

OUT_INCREMENT = 100 # printout after n batches - and save

In [5]:
def cardGenerator(cardtext, windowsize, cards_per_batch, c2i=c2i, debug=False):
    
    i = 0
    indices = list(np.random.permutation(len(cardtext)))
    idx = indices[i]
    
    def nextcard(cardtext, idx, debug=debug):
        if debug:
            card_idx = cardtext[idx]
        else:
            card_idx = [c2i[c] for c in cardtext[idx]]
        
        return list(card_idx)
    
    # pregenerate warmup sequence
    if debug:
        sequence = list(cardtext[idx][-(windowsize):])
    else:
        sequence = list([c2i[c] for c in cardtext[idx][-(windowsize):]])
    i += 1
    idx = indices[i]
    for j in range(cards_per_batch):
        sequence += nextcard(cardtext, idx)
        i += 1
        idx = indices[i]

    # create matrix
    x = []
    y = []
    
    # main iterator
    while True:
        
        # generate batch (of cards_per_batch cards)
        while len(sequence) > windowsize:
            x.append(np.array(sequence[:windowsize]))
            y.append(sequence[windowsize])
            sequence.pop(0)
        
        # generate batch_size worth of window-shifted data
        # reshape for sparse_categorical_crossentropy
        sequence = []
        y = np.array(y)
        y = y[:, np.newaxis]
        # yield and reset
        yield(np.asarray(x), y)
        x, y = [], []
        
        # check for too long, reset
        if len(indices[i:]) < cards_per_batch+1:
            indices = np.random.permutation(len(cardtext))
            i = 0
            idx = indices[i]
        else:
            i += 1
            idx = indices[i]
            
        # pregenerate warmup sequence
        if debug:
            sequence = list(cardtext[idx][-(windowsize):])
        else:
            sequence = list([c2i[c] for c in cardtext[idx][-(windowsize):]])
        i += 1
        idx = indices[i]
        for j in range(cards_per_batch):
            sequence += nextcard(cardtext, idx)
            i += 1
            idx = indices[i]

In [6]:
getbatch = cardGenerator(cardtext, WINDOW_SIZE, CARDS_PER_BATCH)

In [7]:
# define model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE, 
                    batch_input_shape=(1, WINDOW_SIZE, )))
model.add(Dropout(DROP_RATE))
for _ in range(HIDDEN_LAYERS-1):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True, stateful=True))
model.add(LSTM(HIDDEN_SIZE, stateful=True))
model.add(Dense(VOCAB_SIZE, activation='softmax'))

In [8]:
# compile
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [9]:
# predict 'Ⓔ'

def predict(startchars='none', temperature=1.0, maxlen=600):
    
    seq_out = []
    
    if temperature=='random':
        tmp = np.random.random()
    else:
        tmp = temperature
    
    # starting sequence
    if startchars=='none':
        seq_in = [c2i['Ⓔ'] for i in range(WINDOW_SIZE)]
    
    elif startchars=='random':
        seq_in = []
        alpha = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
                 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
                 'w', 'x', 'y', 'z']
        alpha = [a for a in alpha if a in c2i.keys()]
        while len(seq_in) < WINDOW_SIZE-1:
            rnd = np.random.randint(0, len(alpha))
            seq_in += [c2i[alpha[rnd]]]
        seq_in += [c2i['Ⓔ']]
    
    else:
        s = list(startchars)
        s = s[:WINDOW_SIZE]
        seq_out =  [c2i[c] for c in s]
        while len(s) < WINDOW_SIZE:
            s.insert(0, 'Ⓔ')
        seq_in = [c2i[c] for c in s]
        
    # softmax temperature
    # scaling factor of logits = logits/temperature
    # high temp = more confident = more diverse, more mistakes
    # low temp: more conservative
    # https://stackoverflow.com/questions/37246030/how-to-change-the-temperature-of-a-softmax-output-in-keras/37254117#37254117
    def sample(a, temperature=tmp):
        a = np.array(a)**(1/temperature)
        p_sum = a.sum()
        sample_temp = a/p_sum 

        # stupid fix for > 1 error
        while sum(sample_temp) > 1:
            sample_temp[0] -= 0.0001

        return np.argmax(np.random.multinomial(1, sample_temp, 1))

    for i in range(maxlen):

        # predict next char
        pred_out = model.predict(np.array(seq_in).reshape((1, WINDOW_SIZE)))
        # get index of highest pred
        idx = sample(pred_out[0])
        # save index for decoding
        seq_out.append(idx)
        # add index to input sequence
        seq_in.append(int(idx))
        # remove earliest
        seq_in.pop(0)

    # decode final sequence
    card_char = ''.join([i2c[int(i)] for i in seq_out])
    card_char = card_char.replace('Ⓔ', '\n|')
    card_char = card_char.replace('·', '\n')
    card_text = card_char.split('|')

    for f in card_text:
        print(f)
        
    return card_text

In [ ]:
# # load model
model.load_weights('model/v2-modelweights-epoch4-cards1000_start.h5')
START_EPOCH = 4

In [ ]:
# just train, fix epochs later

# save card predictions
predictions = []

for epoch_idx in range(START_EPOCH, NUM_EPOCHS):
    
    # go through all cards each epoch
    for batch in tqdm(range(int(len(cardtext)/CARDS_PER_BATCH))):
        
        # get batch
        x_batch, y_batch = next(getbatch)
        
        # fit to card batch
        r = model.fit(x_batch, y_batch, epochs=1, batch_size=1, shuffle=False, verbose=0)

        # reset state
        model.reset_states()
        
        # if batch % OUT_INCREMENT == 0 and batch > 0:

        if batch % OUT_INCREMENT == 0 and batch > 0:
            model.save_weights('model/v2-modelweights-epoch{}-cards{}.h5'.format(epoch_idx+1, CARDS_PER_BATCH*(batch)))
            print("EPOCH:", epoch_idx+1, "card #:", batch, "of", len(cardtext), r)
            predict(startchars='random', temperature='random')
            predict(startchars='random', temperature='random')
            predict(startchars='random', temperature='random')
            print('\n\n')

  3%|▎         | 100/2934 [44:56<20:43:49, 26.33s/it]

EPOCH: 5 card #: 100 of 29347 <keras.callbacks.History object at 0x7f50a80bc940>
usor ratterer
①Ⓡ
U
sorcery
⌧
destroy  other basic land the next cards from the number of for each creature this abilityp, target creature gets +1/+1 until end of turn.

derile of carm
③Ⓦ
C
creature
elemental
as long as the number of combat creature attacks, sacrifice a creature or player, target creature gets +1/+1 until end of turn.

sereal of wand
③Ⓤ
C
instant
⌧
target creature gets +1/+1 until end of turn.
3
4

sedeare the the of charod
③Ⓤ
C
enchantment
⌧
as a creatures you control a creature this turn, create creature gets +1/+1 until end of turn.
Ⓝ deals 1 damage to target creature you contr
gor of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of the of of of of of of of of of of of of of of of of the of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of the of of of of of of of of of of of of of

  3%|▎         | 101/2934 [45:42<25:28:22, 32.37s/it]

goleremed mory
①Ⓖ
R
creature
human
warrior
flying
whenever this turn, then for each creature this turn, tap to your mana pool. if you do, target creature gets +1/+1 until end of turn.

musoalalester the chale
③Ⓖ
C
creature
human
warrior
enchant creature
enchanted creature gets +1/+1 until end of turn.
↷: Ⓝ gets +1/+1 until end of turn.

se starater
②Ⓖ
R
creature
beast
flying
whenever a creature this turn, then cards.
when Ⓝ spell, then with control three time three spell cards from the number of with it opponent control this turn, sacrifice Ⓝ: target creature gets +1/+1 until end of turn.

cooron





  7%|▋         | 200/2934 [1:29:57<20:15:52, 26.68s/it]

...

 68%|██████▊   | 2000/2934 [15:13:06<7:12:20, 27.77s/it]

EPOCH: 5 card #: 2000 of 29347 <keras.callbacks.History object at 0x7f506fd4ab70>
ke lall
②Ⓖ
U
creature
elemental
sright
flyil
wizard
flying
when Ⓝ enters⑦⑦⑥{

q⑧



•

j⑦7⑦⑦⑦7

⑤

Ⓧ

⑤⑥⑥7

⑥⑥{⑥⑦

⑤⑦

⑥7

{

7⑦Ⓢ⑦

⑤(

7{

7⑦(û⑦⑥⑥⑦

⑥

⑥⑥⑥

⑥⑥

③j78

⑧

{⑥⑦⑦

⑤

⑥⑥j⑦⑥

⑤⑥⑥⑥⑥⑥

③j7

⑤⑦

⑤⑦

⑤⑦



⑤⑥⑥⑥⑥⑥⑥

③⑥

⑥⑥⑥⑥

⑥⑥⑥{;⑬⑮⑳7(⑦⑦⑥7

⑦

⑤⑦

⑥⑥⑦

⑤⑥⑦

⑥{88



j(③7

7{

⑥{

7⑦ûⓈ⑥7

7{

{(68já;7778−⑥⑥;7

{⑥)

{⑦⑦

7

7

q

q⑦;⑬⑳{⑦{687

{

78;7⑥ú⑥

⑥⑤

⑥③

{⑦{⑥;77{7{7777777777777

{7⑥{⑥7

{⑦⑦

7

7

⑥

⑥③

⑥

{⑥Ⓢ

⑥{

7

q

q⑧⑧

q

q

q⑥⑦

⑥③

⑥⑤

⑥③

{⑥{⑦j8

â⑥⑧

⑥⑤

{⑳á⑦⑦

⎕j⑦öú⑬

⑥8

⑥⑤

⑥⑤

⑥⑤

⑦⑤

⑥7

q⑦

⑦



j⑦8⑦⑦

⑦

⑥7

⑥⑦

⑥⑥⑤



;⑦⑥⑥⑦

⑤⑦

;8−⑥⑥{;é⑬8⑥7

{

7

7{

⑦⑦7

7

q{−;7⑥{⑥;!Ⓢé⑥7

q{7{

⑦⑦

⑥q

q

q{⑯âíá{⑬7⑥⑦



⑥;⑦7⑥q

7{R⑩7⑦⑦⑦

⑤⑦

⑥{⑥⑫⑥7

q{7!

7

{⑥⑪é⑦

7

⑤⑥

③j7



③j7⑦⑦

③⑦

⑥{⑥⑮
7⑦7

{

7⑦⑦



⑥⑦

③{⑥⑥7

7

qqq8

q{j{j{⑦8

7⑥7

q

7

q⑥q{

{  ⑦;({

{⑥(j7⑦{

78

7⑥{⑥q{;8

7



{

û⑦7⑦



⑥

7

q

q{

{; ( (8⑥{{7{{⑥7⑥7

Ⓧ

Ⓧ

③

q{

{⑦q

{⑦⑦

q

q{á(  {  ⑧

{  ⑦⑦



{⑥{S7

 68%|██████▊   | 2001/2934 [15:13:42<7:46:19, 29.99s/it]

{⑳⑦⑮7



7{S{77787



③7

q{78



③jⓈ7é⑥{

78

{

7⑦⑮



{⑥q{;(−⑥7

7



•⑥q

{{{{7{77⑥{jq{7⑦{j7⑪⑥{

7⑦⑦⑮



77{j(⑥û⑥7

•

j⑦77

77

7

Ⓧ

7

7

⑥{

7⑦⑦

7⑥{

7⑦Ⓢ

{

7⑦({7ú⑥⑥{

7

{⑥à⑥

7

⑥

⑥⑥

⑥{

{⑫7

;778−⑦

{⑥⑮⑦⑥

⑥{

7

⑥{

7

⑥{

7

7

⑥q

q{j{⑬M⑥7

{

7⑦



⑦

{⑥û⑦

j−é⑥7

7

⑥

⑥⑥

③⑥⑦

q⑥{ú{

7

7

7{

{âö87

{⑥{⑬{⑮Ⓢé77

q

7



•③j⑥{jj5{6j⑦⑩7â



7jjj⑧{6⑦7⑦jé7

{⑩{ö67⑦7⑥7

③{⑥⑥7

⑥

7

7

7

⑥{

7

{⑪⑪â7;77Ⓢ⑥7

{77⑥777757

⑥⑥

③

{⑥é

{⑫á!⑥;7⑥{

7

7{

⑦⑦

⑦

⑥{⑥{⑥{88

{

7⑥{

7⑦j7

⑦⑦

;((á⑥{⑦{7777

{⑥à⑦

{⑥⑥7

7

qq

q⑧

q{⑦q

7

•{⑦7⑦⑦

77

⑥{

7{

⑥{⑥⑥{

7

{⑥q⑥q

ú

7

⑤⑦

⑥j

7⑥{

7

{⑦⑦

⑥{(ö(⑫7⑥⑥{⑦⑦

7

③q



7

•j6{⑦j6öⓈ{

{⑦⑦

7⑦



⑥{

7⑦⑦

③{

7⑦é

{

7⑦⑩⑦

Ⓧ;

;á⑥{⑦⑥⑥⑦

{(7⑥⑥{⑦⑦

7

7

q

q{j{{{⑥7{

jⓈ7({





 69%|██████▉   | 2036/2934 [15:29:28<6:57:39, 27.91s/it]

In [ ]:
# # todo: just save to json one time
# model.save('model/100test_model.h5')
# print("saved model to disk\n")
# model.save_weights('model/100test_model_weights.h5')
# print("saved model weights to disk\n")

In [ ]:
# # load model
# model.load_weights('model/v2-modelweights-epoch4-cards1000.h5')
# START_EPOCH = 25

In [ ]:
predict(startchars='random', temperature='random')